In [116]:
import pandas as pd
import re

# Пример: база данных рецептов

In [90]:
recipes = pd.read_json('recipeitems.json', lines=True) 
recipes.shape  # почти 200 тысяч рецептов и 17 столбцов

(173278, 17)

In [91]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [100]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

Средняя длина списка ингредиентов составляет почти 250 символов при минимальной длине 0 и максимальной — почти 10 000 символов!

In [103]:
# сколько рецептов описывают еду, предназначенную для завтрака
recipes.description.str.contains('[Bb]reakfast').sum()

3524

In [107]:
# сколько рецептов содержат корицу (cinnamon) в списке ингредиентов
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

### Простая рекомендательная система для рецептов

In [119]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley', 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

# создадим булев объект DataFrame, состоящий из значений True и False, указывающих, содержится ли данный ингредиент в списке
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE)) for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [123]:
# хотим найти рецепт, в котором используются петрушка (parsley), паприка (paprika) и эстрагон (tarragon)
selection = spice_df.query('parsley & paprika & tarragon')
print(len(selection))
recipes.name[selection.index]  # выяснить названия этих рецептов

10


2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

In [117]:
recipes[recipes.ingredients.str.contains('parsley') & recipes.ingredients.str.contains('paprika') & recipes.ingredients.str.contains('tarragon')].ingredients.count()

10